In [2]:
from absl import logging

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps
from scipy.spatial import cKDTree
from skimage.feature import plot_matches
from skimage.measure import ransac
from skimage.transform import AffineTransform
from six import BytesIO

import tensorflow as tf

import tensorflow_hub as hub
from six.moves.urllib.request import urlopen

In [3]:
delf = hub.load('https://tfhub.dev/google/delf/1').signatures['default']

In [4]:
def delf_work(path1, path2):
  image1 = Image.open(path1)
  image1 = ImageOps.fit(image1, (96, 96), Image.ANTIALIAS)
  image2 = Image.open(path2)
  image2 = ImageOps.fit(image2, (96, 96), Image.ANTIALIAS)
  result1 = run_delf(image1)
  result2 = run_delf(image2) 

  if (type(result1) is int or type(result2) is int) and (result1==0 or result2 == 0):
    return 0 

  distance_threshold = 0.8

  # Read features.
  num_features_1 = result1['locations'].shape[0]
  #print("Loaded image 1's %d features" % num_features_1)
  
  num_features_2 = result2['locations'].shape[0]
  #print("Loaded image 2's %d features" % num_features_2)

  # Find nearest-neighbor matches using a KD tree.
  d1_tree = cKDTree(result1['descriptors'])
  _, indices = d1_tree.query(
      result2['descriptors'],
      distance_upper_bound=distance_threshold)

  # Select feature locations for putative matches.
  locations_2_to_use = np.array([
      result2['locations'][i,]
      for i in range(num_features_2)
      if indices[i] != num_features_1
  ])
  locations_1_to_use = np.array([
      result1['locations'][indices[i],]
      for i in range(num_features_2)
      if indices[i] != num_features_1
  ])

  # Perform geometric verification using RANSAC.
  try:
    _, inliers = ransac(
        (locations_1_to_use, locations_2_to_use),
        AffineTransform,
        min_samples=3,
        residual_threshold=20,
        max_trials=1000)
  except:
    inliers = [0]

  if inliers is None:
    print('Found 0 inliers')
    return 0
  print('Found %d inliers' % sum(inliers))
  return sum(inliers)


In [5]:
def local_feature(path):
  x = 0
  v=[]
  result = pd.read_csv(path)
  for i in result['id']:
    a = result.loc[result['id'] == i, "landmarks"]
    c = result.loc[result['id'] == i, "confidence"]
    a = a.values
    c = c.values
    #print(a,c)
    if c<0.4:
      p2 = '/content/sample_data/dataset/dataset3/train_images_model/'+str(a[0])
      r = os.listdir(p2)

      if(len(r)>2):
        s=0
        for j in range(2):
          s = max(s,delf_work('/content/sample_data/test_set/'+str(i)+'.jpg', p2+'/'+str(r[j]) ))
        avg = s
      else:
        s=0
        for j in range(len(r)):
          s = max(s,delf_work('/content/sample_data/test_set/'+str(i)+'.jpg', p2+'/'+str(r[j]) ))
        avg = s

    
      if avg < 3:
        result.loc[result.id == i, "confidence"] = 0
      elif avg >= 10:
        result.loc[result.id == i, "confidence"] = 0.6
    elif c>0.4 and c<0.6:
      p2 = '/content/sample_data/dataset/dataset3/train_images_model/'+str(a[0])
      r = os.listdir(p2)

      if(len(r)>2):
        s=0
        for j in range(2):
          s = max(s,delf_work('/content/sample_data/test_set/'+str(i)+'.jpg', p2+'/'+str(r[j]) ))
        avg = s
      else:
        s=0
        for j in range(len(r)):
          s = max(s,delf_work('/content/sample_data/test_set/'+str(i)+'.jpg', p2+'/'+str(r[j]) ))
        avg = s
    elif c==1:
      p2 = '/content/sample_data/dataset/dataset3/train_images_model/'+str(a[0])
      r = os.listdir(p2)

      if(len(r)>5):
        s=0
        for j in range(5):
          s = max(s,delf_work('/content/sample_data/test_set/'+str(i)+'.jpg', p2+'/'+str(r[j]) ))
        avg = s
      else:
        s=0
        for j in range(len(r)):
          s = max(s,delf_work('/content/sample_data/test_set/'+str(i)+'.jpg', p2+'/'+str(r[j]) ))
        avg = s
    
      if avg < 3:
        result.loc[result.id == i, "confidence"] = 0
    #v.append([x,avg,c])
    print(x)
    x+=1


    
  result.to_csv('delf_test_best.csv')
  #return v
  #print(GAP_metric_2(result))

In [ ]:
local_feature("/content/finalensemble.csv")

Streaming output truncated to the last 5000 lines.
10880
10881
Found 0 inliers
Found 0 inliers
10882
Found 4 inliers
Found 0 inliers
10883
Found 0 inliers
Found 7 inliers
10884
Found 17 inliers
Found 9 inliers
10885
Found 11 inliers
Found 11 inliers
10886
10887
Found 0 inliers
Found 0 inliers
10888
Found 9 inliers
Found 5 inliers
10889
Found 0 inliers
Found 0 inliers
10890
10891
Found 5 inliers
Found 8 inliers
10892
Found 22 inliers
Found 11 inliers
10893
10894
10895
Found 0 inliers
Found 0 inliers
10896
Found 8 inliers
Found 0 inliers
10897
Found 15 inliers
Found 12 inliers
10898
Found 0 inliers
Found 0 inliers
10899
10900
Found 7 inliers
Found 6 inliers
10901
Found 17 inliers
Found 5 inliers
10902
Found 0 inliers
Found 0 inliers
10903
10904
10905
Found 0 inliers
Found 0 inliers
10906
Found 42 inliers
Found 5 inliers
10907
Found 0 inliers
Found 0 inliers
10908
Found 24 inliers
Found 11 inliers
10909
10910
10911
Found 6 inliers
Found 0 inliers
10912
Found 0 inliers
Found 0 inliers
1091

In [6]:
# This call calulates the Global Average Precision Metric 
# 
import pandas as pd
def GAP_metric(path):
    result=pd.read_csv(path)
    test=pd.read_csv("/content/drive/My Drive/ADM/test_sample3.csv")
    result=result.sort_values(by='confidence', ascending=False)
    M=0
    true=0
    total=0
    s=0
    relevance=0
    for i in result['id']:
        total+=1
        a=result.loc[result['id'] == i, "landmarks"]
        b=a.values
        #if i in test['id']:
        if i in test.id.values:
          M+=1
          c=test.loc[test['id'] == i, "landmark_id"]
          d=c.values
          if b[0]==d[0]:
              true+=1
              relevance=1
        s+=(true/total)*relevance
        relevance=0
    print(M,s,total,true)
    print(s/M)
    return s/M
GAP_metric("/content/delf_test_best.csv")

5305 2179.42304373032 13067 4921
0.4108243249256022


0.4108243249256022